# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

import numpy as np
from scipy import optimize
from matplotlib import pyplot as plt


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import inauguralproject

# Question 1

BRIEFLY EXPLAIN HOW YOU SOLVE THE MODEL.

In [49]:
# code for solving the model (remember documentation and comments)

# %%
# parameter values as given
m = 1   
v = 10
epsilon = 0.3
tau_0 = 0.4
tau_1 = 0.1 
k = 0.4

#utility function 
def utility(c,v,l,epsilon):
    u_out = np.log(c) - v* (l**(1+1/epsilon)/(1+1/epsilon))
    return u_out

#budget constraint
def budget_constraint(m,w,l,tau_0,tau_1,k):
    b_out = m+ (w*l) - (tau_0*w*l  +tau_1*np.max(w*l-k , 0))
    return b_out

# equilibrium
def equilibrium(l, w, epsilon, v, tau_0, tau_1, k):
    c = budget_constraint(m,w,l,tau_0,tau_1,k)
    return -utility(c,v,l,epsilon)



# call solver
def solve(w, epsilon, v, tau_0, tau_1, k, m): 
    solution = optimize.minimize_scalar(
    equilibrium,  method='bounded',
    bounds=(0, 1), args=(w,epsilon, v, tau_0,tau_1,k))

    # solution
    l_star = solution.x
    c_star = budget_constraint(m, w, l_star,tau_0,tau_1,k)
    u_star = utility(c_star,v,l_star,epsilon)
    return l_star,c_star,u_star

l_star = solve(0.5, epsilon, v , tau_0, tau_1, k, m)[0]

c_star = solve(0.5, epsilon, v , tau_0, tau_1, k, m)[1]    

u_star = solve(0.5, epsilon, v , tau_0, tau_1, k, m)[2]        


print("The optimal units of labor is: " + str(l_star))
print("The optimal units of consumption is: " + str(c_star))
print("The maximum utility is: " + str(u_star))

The optimal units of labor is: 0.31961536193545265
The optimal units of consumption is: 1.119903840483863
The maximum utility is: 0.09677772523865749


# Question 2

ADD ANSWER.

In [51]:
import matplotlib.pyplot as plt
plt.style.use("seaborn")

# set of vector values
N = 10000
wage_range = np.linspace(0.5, 1.5, N)
labour_range = np.empty(N)
consumption_range = np.empty(N)

# plot definition
for i, w in enumerate(wage_range):
    sol = solve(w, epsilon, v, tau_0, tau_1, k, m)
    labour_range[i] = sol[0]
    consumption_range[i] = sol[1]


figure = plt.figure(figsize=(10,4))


# left plot
ax_left = figure.add_subplot(1,2,1)
ax_left.plot(wage_range, labour_range)

ax_left.set_title("Optimal Labour")
ax_left.set_xlabel("Wage Rate")
ax_left.set_ylabel("Labour")
#  ax_left.set_ylim([0,1])
#  ax_left.set_xlim([np.min(w),np.max(w)])
ax_left.grid(True)
# right plot
ax_right = figure.add_subplot(1,2,2)
ax_right.plot(wage_range, consumption_range)

ax_right.set_title("Optimal Consumption")
ax_right.set_xlabel("Wage Rate")
ax_right.set_ylabel("Consumption")
## ax_right.set_ylim([1,2])
## ax_right.set_xlim([np.min(w),np.max(w)])
ax_right.grid(True)

# call plot

print(plt.show)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<function show at 0x000001C426495EE8>


# Question 3

ADD ANSWER.

In [53]:
def total_tax(tau_0, wage_range, labour_range, tau_1, k):


    total_tax_revenue = np.sum(tau_0 * wage_range * labour_range + tau_1*np.max (wage_range*labour_range-k,0))
    return total_tax_revenue
print(f'The total tax rate is =  {total_tax(tau_0, wage_range, labour_range, tau_1, k):.2f}')

The total tax rate is =  1775.39


# Question 4

ADD ANSWER.

In [54]:
labour_range_updated = np.empty(N)
consumption_range_updated = np.empty(N)
epsilon_updated = 0.1 
for i, w in enumerate(wage_range):
    sol = solve(w, epsilon_updated, v, tau_0, tau_1, k, m)
    labour_range_updated[i] = sol[0]
    consumption_range_updated[i] = sol[1]

###total_tax_revenue_updated = np.sum(tau_0 * wage_range * labour_range_updated + tau_1*np.max(wage_range*labour_range_updated-k,0))
####print(f'The total tax rate is =  {total_tax_revenue_updated:.2f}')  


## 
print(f'The total tax rate is =  {total_tax(tau_0, wage_range, labour_range_updated, tau_1, k):.2f}')


The total tax rate is =  3578.90
The total tax rate is =  3578.90


# Question 5

ADD ANSWER.

In [57]:

def taxmax(vec):

    tau_0 = vec[0]
    tau_0 = vec[1]
    k = vec[2]

    return  -total_tax(tau_0, wage_range, labour_range, tau_1, k)


# Conclusion

ADD CONCISE CONLUSION.